# 部署活动和筛选器<a class="anchor" id="top"></a>

在本笔记本中，您将在 Amazon Personalize 中部署活动并与之交互。

1. [简介](#intro)
1. [创建活动](#create)
1. [与活动交互](#interact)
1. [批量推荐](#batch)
1. [总结](#wrapup)

## 简介<a class="anchor" id="intro"></a>
[返回页首](#top)

此时，您应该有多个解决方案，每个解决方案至少有一个解决方案版本。一旦创建了解决方案版本，就可以从这些版本中获得推荐，并了解整体表现。

本笔记本首先将以前笔记本中的每个解决方案版本部署到各个活动中。一旦它们处于活跃状态，就有用于查询推荐的资源，并有辅助函数将输出内容转化为更易于阅读的内容。

与 Amazon Personalize 上您的客户一样，您可以修改辅助函数以适应数据输入文件的结构，从而使附加呈现工作正常进行。

要开始工作，我们需要再次导入库，加载以前笔记本中的值，并加载 SDK。

In [1]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import botocore
from botocore.exceptions import ClientError
import pandas as pd

In [2]:
%store -r

In [3]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## 创建活动<a class="anchor" id="create"></a>
[返回页首](#top)

活动是托管解决方案版本；可以查询推荐的端点。通过估计吞吐量（用户每秒的个性化请求）来设置定价。部署活动时，您可以设置每秒最小吞吐量（TPS）值。与 AWS 中的许多服务一样，此服务将根据需求自动扩展，但如果延迟至关重要，您可能希望提前预置以满足更大的需求。对于此 POC 和演示，所有最小吞吐量阈值均设置为 1。有关更多信息，请参阅[定价页面](https://aws.amazon.com/personalize/pricing/)。

让我们开始部署活动。

### 用户个性化

为您的用户个性化解决方案版本部署活动。部署一个活动大约需要 10 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，并且这里的目标是创建多个活动。因此，我们将在笔记本中进一步为所有活动设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [4]:
userpersonalization_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-userpersonalization",
    solutionVersionArn = userpersonalization_solution_version_arn,
    minProvisionedTPS = 1
)

userpersonalization_campaign_arn = userpersonalization_create_campaign_response['campaignArn']
print(json.dumps(userpersonalization_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-userpersonalization",
  "ResponseMetadata": {
    "RequestId": "ea0b63f5-9efa-4abd-bc88-dd5212e85668",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "ea0b63f5-9efa-4abd-bc88-dd5212e85668",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS

为您的 SIMS 解决方案版本部署活动。部署一个活动大约需要 10 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，并且这里的目标是创建多个活动。因此，我们将在笔记本中进一步为所有活动设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [5]:
sims_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-SIMS",
    solutionVersionArn = sims_solution_version_arn,
    minProvisionedTPS = 1
)

sims_campaign_arn = sims_create_campaign_response['campaignArn']
print(json.dumps(sims_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-SIMS",
  "ResponseMetadata": {
    "RequestId": "8d8f06e3-2699-4bcd-bb60-9389aff0fcf4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "8d8f06e3-2699-4bcd-bb60-9389aff0fcf4",
      "content-length": "90",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 个性化排名

为您的个性化排名解决方案版本部署活动。部署一个活动大约需要 10 分钟。通常，我们会使用循环进行轮询，直到任务完成。但是，该任务会阻止其他单元格执行，并且这里的目标是创建多个活动。因此，我们将在笔记本中进一步为所有活动设置 While 循环。在那里，您还可以找到在 AWS 控制台中查看进度的说明。

In [6]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "c28bff59-c214-4f57-85d4-5e2643a03413",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Thu, 17 Sep 2020 00:41:29 GMT",
      "x-amzn-requestid": "c28bff59-c214-4f57-85d4-5e2643a03413",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 查看活动创建状态

如前面所述，如何在控制台中查看状态更新呢：

* 在另一个浏览器选项卡中，您应该已经在打开此笔记本实例时启动了 AWS 控制台。
* 切换到该选项卡并在顶部搜索服务 `Personalize`，然后转到该服务页面。
* 单击 `View dataset groups`。
* 单击数据集组的名称，名称中很可能包含 POC。
* 单击 `Campaigns`。
* 现在，您将看到上面创建的所有活动的列表，包括一个活动状态列。处于 `Active` 状态后，就可以查询您的活动了。

或者，只需运行下面的单元格即可跟踪活动创建状态。

In [7]:
in_progress_campaigns = [
    userpersonalization_campaign_arn,
    sims_campaign_arn,
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-SIMS
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-userpersonalization
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:136455442858:campaign/personalize-poc-rerank


## 创建筛选器<a class="anchor" id="interact"></a>
[返回页首](#top)

现在，所有活动都已部署并处于活跃状态，我们可以创建筛选器。我们可以为项目和事件创建筛选器。视频点播中筛选器的几个常见使用案例是：

基于项目元数据的分类筛选器 – 通常，您的项目元数据将包含有关您的标题的信息，如流派、关键字、年份、年代等 对这些数据进行筛选可以在这些数据范围内提供推荐，例如动作片。

事件 – 您可能希望筛选出某些事件并根据这些事件提供结果，例如将标题从"观看建议"推荐移动至"再次观看"推荐。

让我们来看看项目元数据和用户交互，以了解我们可以创建哪种类型的筛选器。

In [8]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)
#interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
items_df.head(10)
#interactions_df.head(10)

,GENRE,YEAR
ITEM_ID,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995
2,Adventure|Children|Fantasy,1995
3,Comedy|Romance,1995
4,Comedy|Drama|Romance,1995
5,Comedy,1995
6,Action|Crime|Thriller,1995
7,Comedy|Romance,1995
8,Adventure|Children,1995
9,Action,1995


现在我们要做的是确定要筛选的流派，因为我们需要一个包含所有流派的列表。首先，我们将获得 GENRE 列的所有唯一值，然后在`|`（如果存在）上拆分字符串，每个流派都将被添加到一个长列表中，该列表将被转换为一个集以提高效率。然后，该集将被添加到列表中，以便进行迭代，然后我们可以使用创建筛选器 API。

In [9]:
unique_genre_field_values = items_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [10]:
genres_to_filter

['Animation',
 'Thriller',
 'Sci-Fi',
 'IMAX',
 'Horror',
 'Comedy',
 'Adventure',
 'Musical',
 'Drama',
 'Romance',
 'Fantasy',
 'War',
 'Documentary',
 'Crime',
 'Western',
 'Film-Noir',
 'Mystery',
 'Action',
 'Children']

在此之后，我们现在拥有数据集中存在的所有流派。目前，Personalize 的软限制是总共 10 个筛选器。鉴于我们有大量流派，我们将随机选择 7 个筛选器，以便在以后为基于交互的 2 个筛选器留出空间，并为基于年份的推荐额外添加 1 个筛选器

In [11]:
genres_to_filter = random.sample(genres_to_filter, 7)
genres_to_filter

['Film-Noir', 'Fantasy', 'Western', 'Documentary', 'Comedy', 'Action', 'IMAX']

现在，为元数据流派筛选器创建一个列表，然后使用下面的单元格创建实际筛选器。请注意，此过程需要几分钟时间才能完成。

In [12]:
# Create a list for the filters:
meta_filter_arns = []

In [13]:
# Iterate through Genres
for genre in genres_to_filter:
    # Start by creating a filter
    try:
        createfilter_response = personalize.create_filter(
            name=genre,
            datasetGroupArn=dataset_group_arn,
            filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
        )
        # Add the ARN to the list
        meta_filter_arns.append(createfilter_response['filterArn'])
        print("Creating: " + createfilter_response['filterArn'])
    
    # If this fails, wait a bit
    except ClientError as error:
        # Here we only care about raising if it isnt the throttling issue
        if error.response['Error']['Code'] != 'LimitExceededException':
            print(error)
        else:    
            time.sleep(120)
            createfilter_response = personalize.create_filter(
                name=genre,
                datasetGroupArn=dataset_group_arn,
                filterExpression='INCLUDE ItemID WHERE Items.GENRE IN ("'+ genre +'")'
            )
            # Add the ARN to the list
            meta_filter_arns.append(createfilter_response['filterArn'])
            print("Creating: " + createfilter_response['filterArn'])

Creating: arn:aws:personalize:us-east-1:136455442858:filter/Film-Noir
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Fantasy
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Western
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Documentary
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Comedy
Creating: arn:aws:personalize:us-east-1:136455442858:filter/Action
Creating: arn:aws:personalize:us-east-1:136455442858:filter/IMAX


我们再来为已观看和未观看的内容创建 2 个事件筛选器

In [14]:
# Create a dataframe for the interactions by reading in the correct source CSV
interactions_df = pd.read_csv(data_dir + '/interactions.csv', sep=',', index_col=0)

# Render some sample data
interactions_df.head(10)

,ITEM_ID,TIMESTAMP,EVENT_TYPE
USER_ID,,,
429,222,828124615,watch
429,227,828124615,click
429,595,828124615,watch
429,592,828124615,watch
429,590,828124615,watch
429,434,828124615,watch
429,421,828124615,watch
429,225,828124615,click
429,343,828124615,click


我们再来为已观看和未观看的内容创建 2 个事件筛选器

In [15]:
createwatchedfilter_response = personalize.create_filter(name='watched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )

createunwatchedfilter_response = personalize.create_filter(name='unwatched',
    datasetGroupArn=dataset_group_arn,
    filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("watch")'
    )


最后，由于我们现在在项目元数据中提供了年份，让我们创建年代筛选器，仅推荐在给定年代中发布的电影。在本次研讨会中，我们将选择 20 世纪 70 年代的院线电影。 

In [16]:
createdecadefilter_response = personalize.create_filter(name='1970s',
    datasetGroupArn=dataset_group_arn,
    filterExpression='INCLUDE ItemID WHERE Items.YEAR >= 1970 AND Items.YEAR < 1980'
    )

最后一步，我们还需要将这些筛选器添加到列表中，以便以后可以使用它们。

In [17]:
interaction_filter_arns = [createwatchedfilter_response['filterArn'], createunwatchedfilter_response['filterArn']]

In [18]:
decade_filter_arns = [createdecadefilter_response['filterArn']]

In [19]:
%store sims_campaign_arn
%store userpersonalization_campaign_arn
%store rerank_campaign_arn
%store meta_filter_arns
%store interaction_filter_arns
%store decade_filter_arns


Stored 'sims_campaign_arn' (str)
Stored 'userpersonalization_campaign_arn' (str)
Stored 'rerank_campaign_arn' (str)
Stored 'meta_filter_arns' (list)
Stored 'interaction_filter_arns' (list)
Stored 'decade_filter_arns' (list)


您已全部设定为移至最后一个探索性笔记本：`05_Interacting_with_Campaigns_and_Filters.ipynb`。在浏览器中打开它，您可以开始与活动交互并获取推荐！